# Ensemble Model


----
```
: 26.05.24
: zachcolinwolpe@gmail.com
```
----

In [2]:
# importing neccessary libraries 
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, BertPreTrainedModel, BertModel,AdamW
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import pandas as pd 
import numpy as np
import torch
import nltk
nltk.download('stopwords')

from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from collections import Counter
import sklearn


from nltk.corpus import stopwords
import argparse
import logging
import string
import re
import os

from sklearn.model_selection import train_test_split, KFold
import torch.optim as optim
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from transformers import BertModel
import os


/Users/zachwolpe/miniforge3/envs/mlxgo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zachwolpe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from Roberta_classifier import (RobertaWithActivationAndRegularization, RoBertaTokenizer)
from ML_training_code import (generate_K_Fold_data, torch_tensorize, plot_training_validation, training_loop)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zachwolpe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from Bert_classifier import (BertWithActivationAndRegularization, Bert_tokenize)
from ML_training_code import (generate_K_Fold_data, torch_tensorize, plot_training_validation, training_loop)


---
## Load Pre-Trained Models
----

In [19]:
# Config ---------------------------------------->>
PATH_TO_DATA = '../data/train_test/'
SAVE_LOC = '../model-artifacts/'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEBUG_MODE = True
BATCH_SIZE = 10
# Config ---------------------------------------->>

# Load data
X_test = pd.read_csv(PATH_TO_DATA + 'X_test.csv')
y_test = pd.read_csv(PATH_TO_DATA + 'y_test.csv')


# Downsample for testing
if DEBUG_MODE:
    X_test = X_test[:3]
    y_test = y_test[:3]

----
# Load Models
----

In [20]:

# load Bert model
Bert_model = BertWithActivationAndRegularization(pretrained_model_name='bert-base-uncased', num_labels=3)
Bert_model.load_state_dict(torch.load(f'{SAVE_LOC}bert_model.pth'))


# Load RoBerta model
RoBerta_model = RobertaWithActivationAndRegularization(pretrained_model_name='roberta-base', num_labels=3,)
RoBerta_model.load_state_dict(torch.load(f'{SAVE_LOC}roberta_model.pth'))



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

----
# Inference Engine
----

In [31]:

def inference(model, X_test, y_test, tokenizer=Bert_tokenize, criterion=nn.CrossEntropyLoss()):
    """
    Perform inference on the model

    Return softmax probabilities, predictions, and metrics
    """    
    # evaluate on test data
    model.eval()
    input_ids_test, attention_masks_test, y_test = tokenizer(X_test, y_test)
    test_dataset = TensorDataset(input_ids_test, attention_masks_test, y_test)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

    total_test_loss = 0
    correct_test = 0
    total_test = 0

    #return items
    softmax_prob = []
    predictions = []
    accuracy = []

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            labels = labels.reshape(-1)  # Reshape labels once

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs
            loss = criterion(logits, labels)
            total_test_loss += loss.item()

            _, predicted = torch.max(logits, 1)
            correct_test += (predicted == labels).sum().item()
            total_test += labels.size(0)
            accuracy_test = correct_test / total_test
            print(f'Test Loss: {total_test_loss:.4f} - Test Accuracy: {accuracy_test:.4f}')

            # Get softmax probabilities
            softmax_prob.append(nn.functional.softmax(logits, dim=1).cpu().numpy())
            predictions.append(predicted.cpu().numpy())
            accuracy.append(accuracy_test)

        # flatten the list
        softmax_prob = np.concatenate(softmax_prob)
        predictions = np.concatenate(predictions)
        accuracy = np.mean(accuracy)
        
        return softmax_prob, predictions, accuracy



----
# Model Evaluation: Bert 
----

In [32]:
(softmax_prob_Bert,
 predictions_Bert,
 accuracy_Bert) = inference(Bert_model,
                            X_test,
                            y_test,
                            tokenizer=Bert_tokenize)
softmax_prob_Bert, predictions_Bert, accuracy_Bert



Test Loss: 0.9103 - Test Accuracy: 0.6667


(array([[0.61952966, 0.14688203, 0.23358834],
        [0.29358956, 0.47601697, 0.23039342],
        [0.14984523, 0.2641538 , 0.586001  ]], dtype=float32),
 array([0, 1, 2]),
 0.6666666666666666)

In [26]:
softmax_prob
accuracy

[0.6666666666666666]

----
# Model Evaluation: RoBerta 
----

In [30]:
(softmax_prob_RoBerta,
 predictions_RoBerta,
 accuracy_RoBerta) = inference(RoBerta_model,
                            X_test,
                            y_test,
                            tokenizer=RoBertaTokenizer)
softmax_prob_RoBerta, predictions_RoBerta, accuracy_RoBerta



Test Loss: 0.9103 - Test Accuracy: 0.6667


(array([[0.61952966, 0.14688203, 0.23358834],
        [0.29358956, 0.47601697, 0.23039342],
        [0.14984523, 0.2641538 , 0.586001  ]], dtype=float32),
 array([0, 1, 2]),
 0.6666666666666666)

----
# Ensemble Model
----

Ensemble methods:

To ensemble your two deep learning classifiers (Bert and RoBerta models), you can average their predictions. Here's a concise guide on how to implement this in PyTorch:
1. Load Both Models: Ensure both models are loaded and set to evaluation mode.
2. Prepare the Data: Tokenize and prepare your data as required by both models. Since you're using different tokenizers for Bert and RoBerta, you'll need to prepare the inputs separately for each model.
3. Get Predictions from Both Models: Pass the test data through both models to get the logits (raw model outputs before applying softmax).
4. Average the Logits: Average the logits obtained from both models. This simple method of ensembling is often quite effective.
5. Apply Softmax and Determine Final Predictions: Apply softmax to the averaged logits to get probabilities, and then determine the final predictions (e.g., by taking the argmax).
Here's how you could implement this:



This code snippet averages the predictions from both models. You can also explore other ensembling techniques like weighted averaging or voting based on validation set performance.

In [51]:
def ensemble_inference(model1, model2, X_test, y_test, tokenizers=[Bert_tokenize, RoBertaTokenizer], criterion=nn.CrossEntropyLoss()):
    """
    Perform inference on the model

    Return softmax probabilities, predictions, and metrics
    """    
    # Assuming model_bert and model_roberta are your loaded models
    model1.eval()
    model2.eval()

    # Prepare data for both models
    input_ids_bert, attention_masks_bert, _ = tokenizers[0](X_test, y_test)
    input_ids_roberta, attention_masks_roberta, _ = tokenizers[1](X_test, y_test)

    # Convert to appropriate device
    input_ids_bert, attention_masks_bert = input_ids_bert.to(device), attention_masks_bert.to(device)
    input_ids_roberta, attention_masks_roberta = input_ids_roberta.to(device), attention_masks_roberta.to(device)

    # Get logits from both models
    with torch.no_grad():
        logits_model1 = model1(input_ids=input_ids_bert, attention_mask=attention_masks_bert)
  
        logits_model2 = model2(input_ids=input_ids_roberta, attention_mask=attention_masks_roberta)

        # Average the logits
        averaged_logits = (logits_model1 + logits_model2) / 2

        # Apply softmax to get probabilities
        probabilities = torch.nn.functional.softmax(averaged_logits, dim=1)

        # Get the predicted classes
        _, predicted_classes = torch.max(probabilities, 1)

        # Calculate accuracy
        acc = torch.sum(predicted_classes == torch.tensor(y_test['Sentiment'])) / len(y_test)

        return probabilities.cpu().numpy(), predicted_classes.cpu().numpy(), acc.item()

        

In [52]:
# Ensemble Inference
(softmax_prob_ensemble,
 predictions_ensemble,
 accuracy_ensemble) = ensemble_inference(Bert_model,
                                        RoBerta_model,
                                        X_test,
                                        y_test,
                                        tokenizers=[Bert_tokenize, RoBertaTokenizer])


(softmax_prob_ensemble,
 predictions_ensemble,
 accuracy_ensemble) 

(array([[0.4423466 , 0.21856269, 0.33909073],
        [0.33239466, 0.40170312, 0.26590228],
        [0.200824  , 0.2807705 , 0.51840544]], dtype=float32),
 array([0, 1, 2]),
 0.6666666865348816)

In [53]:
# # evaluate on test data
# Bert_model.eval()
# criterion = nn.CrossEntropyLoss()
# input_ids_test, attention_masks_test, y_test = Bert_tokenize(X_test, y_test)
# test_dataset = TensorDataset(input_ids_test, attention_masks_test, y_test)
# test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# total_test_loss = 0
# correct_test = 0
# total_test = 0

# with torch.no_grad():
#     for batch in test_dataloader:
#         input_ids, attention_mask, labels = batch
#         input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
#         labels = labels.reshape(-1)  # Reshape labels once

#         outputs = Bert_model(input_ids=input_ids, attention_mask=attention_mask)
#         logits = outputs
#         loss = criterion(logits, labels)
#         total_test_loss += loss.item()

#         _, predicted = torch.max(logits, 1)
#         correct_test += (predicted == labels).sum().item() 
#         total_test += labels.size(0)
#         accuracy_test = correct_test / total_test
#         print(f'Test Loss: {total_test_loss:.4f} - Test Accuracy: {accuracy_test:.4f}')


In [14]:

# Evaluate on test data
RoBerta_model.eval()
criterion = nn.CrossEntropyLoss()
input_ids_test, attention_masks_test, y_test = RoBertaTokenizer(X_test, y_test)
test_dataset = TensorDataset(input_ids_test, attention_masks_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

total_test_loss = 0
correct_test = 0
total_test = 0

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        labels = labels.reshape(-1)  # Reshape labels once

        outputs = RoBerta_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs
        loss = criterion(logits, labels)
        total_test_loss += loss.item()

        _, predicted = torch.max(logits, 1)
        correct_test += (predicted == labels).sum().item() 
        total_test += labels.size(0)
        accuracy_test = correct_test / total_test
        print(f'Test Loss: {total_test_loss:.4f} - Test Accuracy: {accuracy_test:.4f}')
        

IndexError: too many indices for tensor of dimension 1

In [ ]:
# Assuming model_bert and model_roberta are your loaded models
model_bert.eval()
model_roberta.eval()

# Prepare data for both models
input_ids_bert, attention_masks_bert, _ = Bert_tokenize(X_test, y_test)
input_ids_roberta, attention_masks_roberta, _ = RoBertaTokenizer(X_test, y_test)

# Convert to appropriate device
input_ids_bert, attention_masks_bert = input_ids_bert.to(device), attention_masks_bert.to(device)
input_ids_roberta, attention_masks_roberta = input_ids_roberta.to(device), attention_masks_roberta.to(device)

# Get logits from both models
with torch.no_grad():
    outputs_bert = model_bert(input_ids=input_ids_bert, attention_mask=attention_masks_bert)
    logits_bert = outputs_bert.logits  # Adjust depending on your model's output

    outputs_roberta = model_roberta(input_ids=input_ids_roberta, attention_mask=attention_masks_roberta)
    logits_roberta = outputs_roberta.logits  # Adjust depending on your model's output

    # Average the logits
    averaged_logits = (logits_bert + logits_roberta) / 2

    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(averaged_logits, dim=1)

    # Get the predicted classes
    _, predicted_classes = torch.max(probabilities, 1)

# Evaluate the predictions as needed